In [3]:
import pandas as pd
from tqdm import tqdm
#from RM_tools import dp_utils
from pathlib import Path
import pickle 
import datetime
from calendar import month_abbr, different_locale

In [353]:
# %% load dicts
features_group = pd.read_excel(r'.\codes\feature_groups.xlsx')
demo_order = pd.read_excel(r'.\codes\demo_order.xlsx')
demo_order['buyers_gr_label'] = demo_order['buyers_gr_label'].astype(str).str.lower()
segments_order = pd.read_excel(r'.\codes\segments_order.xlsx')
example = pd.read_excel(r'.\codes\Example.xlsx', sheet_name = 'example')
legends = pd.read_excel(r'.\codes\Example.xlsx', sheet_name = 'legends')
shop_order = pd.read_excel(r'.\codes\shop_order.xlsx')
today = datetime.datetime.now().strftime('%Y-%m-%d')


list_of_category = ['Hair Conditioners',
 'Laundry Detergents',
 'MALE B&R',
 'Shampoos',
 'Feminine Care',
 'Diapers']

#get_low!
list_of_dict = [features_group,demo_order,
                 segments_order,example,legends,shop_order]
for d in list_of_dict:
    d.columns = [c.lower() for c in d.columns ]

fin_cols = example.columns.to_list() + ['shop_code', 'shop_lvls','channel_code', 'file']
columns_dict = legends[['mask','Colum'.lower()]].dropna(subset='mask').set_index('mask').to_dict()['Colum'.lower()]



def get_category(name_of_file, list_of_category=False):
    cat_dict = {'br': 'MALE B&R',
 'deterg': 'Laundry Detergents',
 'diapers': 'Diapers',
 'femcare': 'Feminine Care',
 'haircond': 'Hair Conditioners',
 'shampoo': 'Shampoos',
 'MALE B&R': 'MALE B&R',
 'Laundry Detergents': 'Laundry Detergents',
 'Diapers': 'Diapers',
 'Feminine Care': 'Feminine Care',
 'Hair Conditioners': 'Hair Conditioners',
 'Shampoos': 'Shampoos'}
    for k in cat_dict.keys():
        if name_of_file.count(k):
            return cat_dict[k]
    else:
        raise ValueError('В названии файла нет категории', name_of_file)    


# %% make_functions

def get_period_lbl(time_period_type, year, month):
    with different_locale('English'):
        return f"{time_period_type.split('/')[0]}: {month_abbr[month]} {year}"

def get_label_num(time_period_type, year, month):
    with different_locale('English'):
        return f"{time_period_type.split('/')[0]}: {year} ({str(month).zfill(2)}) {month_abbr[month]}"


def add_period_lbls(df_in):
    periods = pd.read_excel(r'.\codes\periods.xlsx', sheet_name ='period_lbl').drop(columns='Unnamed: 0')
    time_period_type = pd.read_excel(r'.\codes\periods.xlsx', sheet_name ='time_period_type')


    time_period_type_dict = {24 :'2MATs/ 104 we', 12:'MAT/ 52 we', 3:'3MMT/ 12we', 1: 'Month'}
    
    col_to_int = ['duration', 'year', 'month']
    df_in[col_to_int] = df_in[col_to_int].astype(int)
    
    df_in['time_period_type'] = df_in['duration'].map(time_period_type_dict)
    
    df_tmp_per = df_in[['time_period_type', 'year', 'month']].drop_duplicates()
    df_tmp_per['period_lbl'] = df_tmp_per[['time_period_type', 'year', 'month']]\
        .apply(lambda row: get_period_lbl(*row), axis=1)
    
    df_tmp_per['label_num'] = df_tmp_per[['time_period_type', 'year', 'month']]\
        .apply(lambda row: get_label_num(*row), axis=1)
    


    new_periods = set(df_tmp_per['period_lbl']) - set(periods['period_lbl'])
    if len(new_periods):
        print('New periods!')
        print(*new_periods)
        new_periods_df = df_tmp_per[df_tmp_per['period_lbl'].isin(new_periods)].sort_values(by=['year','month'])
        new_periods_df['date_added'] = today
        periods = pd.concat([periods[['period_lbl','label_num','date_added' ]], new_periods_df[['period_lbl','label_num','date_added' ]]],ignore_index=True )
        periods['period_code'] = range(1,len(periods)+1)


        with pd.ExcelWriter(r'.\codes\periods.xlsx',engine="openpyxl",mode="a", if_sheet_exists='replace') as writer:
            periods.to_excel(writer, sheet_name ='period_lbl')





    df_in = df_in.merge(df_tmp_per,on = ['time_period_type', 'year', 'month'], how='left')
    
    df_in = df_in.merge(
            periods, on=['period_lbl', 'label_num'],  how='left', validate='many_to_one')
    
    
    
    df_in = df_in.merge(
           time_period_type, on='time_period_type', how='left', validate='many_to_one')

    df_in = df_in.drop(columns=['time_period_type', 'period_lbl', 'month' ])\
            .rename(columns={'time_period_code': 'time_period_type', 'period_code': 'period_lbl'})

    return df_in

def get_df_in_v2(df_in, category, columns_dict):
    
    #get_low_2!
    df_in.columns = [c.lower() for c in df_in.columns]
    income_columns = list(df_in.columns)


    df_in = df_in.rename(
        {'Category Name'.lower():'Product Name'.lower(), 'Buyer Group Name'.lower():'buyers_gr_label'}, 
        axis=1)
    df_in['Category Name'.lower()] = category
    df_in['buyers_gr_label'] = df_in['buyers_gr_label'].replace({'Total Population' :'TOTAL DEMOGRAPHICS'})
    df_in = df_in.drop('category',axis=1)


    #print('init shape', df_in.shape)


        
    # products
    #prod_col_to_merge = 'Product Name'.lower()
    #if  len(set(df_in['Product Name'.lower()].unique()) - set(features_group['Product Name'.lower()]))>0:
    #    print('Мерджим по фулл нейму')
    #    prod_col_to_merge = 'full_label'
    #    features_group = features_group.drop(columns='Product Name'.lower())
    prod_col_to_merge = 'full_label'

    
    df_in = df_in.merge(
        features_group, 
        left_on=['Category Name'.lower(), 'Product Name'.lower()], 
        right_on=['Category Name'.lower(),  prod_col_to_merge], 
        how='left', validate='many_to_one') #Есть множественные ключи в features_group['Product Name']
    




    df_in = df_in.rename(
        columns={'product_code': 'products', 'category_code': 'category', 'feature_code': 'features'}
    )
    df_in['product_hier'] = df_in['products']


    # socdem
    df_in['buyers_gr_label'] = df_in['buyers_gr_label'].str.lower()
    df_in = df_in.merge(
        demo_order, on=['buyers_gr_label'], how='left', validate='many_to_one')
    df_in = df_in.rename(columns={'demo_code': 'demo_groups'})
    df_in['demo_hier'] = df_in['demo_groups']

    
    # segments

    if ('Segment'.lower() not in df_in.columns):
        if (category == 'Feminine Care'):
            df_in['Segment'.lower()] = 'Total Feminine Care'
        elif (category == 'Shampoos'):
            df_in['Segment'.lower()] = 'Shampoos'
    

    
    if (category == 'Laundry Detergents'):
        df_in['Segment'.lower()] = df_in['Segment'.lower()].replace(
            {'Total Detergents excluding Bars': 'Total Detergents excluding Bar'})
    df_in['Segment'.lower()] =df_in['Segment'.lower()].replace({'Total Diapers size':'Total Diapers'})
    df_in = df_in.merge(
        segments_order, on=['Segment'.lower()], how='left', validate='many_to_one')
    df_in = df_in.rename(columns={'segment_code': 'cat_segment'})


    # shop
    df_in['position_name_shop'] = df_in['position_name_shop'].replace(
    {'Ok Hyper': 'OK Hyper', 'Perekryostok': 'Perekrestok'})
    df_in = df_in.merge(
        shop_order, on=['position_name_shop'], how='left', validate='many_to_one')

    
    #print('merged shape', df_in.shape)
    
    # rename metrics
    df_in.columns = [col.lower().replace(' ','_') for col in df_in.columns]
    df_in = df_in.rename(columns_dict, axis=1)
    df_in = df_in[[i for i in df_in.columns if i in fin_cols + ['duration','month'] + income_columns ]]
    
    return df_in
    
def get_buyers_shares(df_in): # %% value / buyers shares for socdem
    cols = ['product_lvls', 'category',
        'shop_code', 'shop_lvls',
        'time_period_type', 'year', 'period_lbl',
        'product_hier', 'products', 
        'features', 'cat_segment'] 
    cols_to_rename = {'spend_local_currency': 'total_spend_local_currency', 
                'buying_households': 'total_buying_households'}
    cols_to_merge = list(df_in.columns[df_in.dtypes != 'float64'].difference([*demo_order.columns, 'demo_groups']))

        
    totals = df_in.loc[df_in['demo_groups'] == 1,[*cols_to_merge,'spend_local_currency','buying_households']].rename(columns=cols_to_rename)
    totals = totals.drop_duplicates()
    df_in = df_in.merge(totals, how='left',  validate='many_to_one')

    df_in['value_share'] = df_in['spend_local_currency'] / df_in['total_spend_local_currency'] * 100
    df_in['buyers_share'] = df_in['buying_households'] / df_in['total_buying_households'] * 100


    df_in = df_in.drop(columns=['total_spend_local_currency', 'total_buying_households'])
    
    return df_in


In [23]:
#%%
import pandas as pd
from tqdm import tqdm
#from RM_tools import dp_utils
from pathlib import Path
import pickle 
chunk_size = 1_000_000


# %% load old format data

files = Path('./data/tmp').glob('*.pq')
dfs = []
for file in files:
    print(file.stem)
    tmp = pd.read_parquet(file)
    print(tmp.shape)
    
    dfs.append(tmp)


df_union = pd.concat(dfs, ignore_index=True)

del dfs

print("Creating united data file complite!",df_union.shape )


# %% value / buyers shares for socdem
# cols = ['product_lvls', 'category',
#         'shop_code', 'shop_lvls',
#         'time_period_type', 'year', 'period_lbl',
#         'product_hier', 'products', 
#         'features', 'cat_segment','demo_hier'] 
# cols_to_rename = {'spend_local_currency': 'total_spend_local_currency', 
#              'buying_households': 'total_buying_households'}
        
# totals = df_union.loc[df_union['demo_groups'] == 1, cols+ [*cols_to_rename.keys()]].rename(
#     columns={'spend_local_currency': 'total_spend_local_currency', 
#              'buying_households': 'total_buying_households'})
# totals = totals.drop_duplicates()

# df_union = df_union.merge(totals, how='left',left_on =cols + [*cols_to_rename.keys()], right_on =cols +[*cols_to_rename.values()],  validate='many_to_one')

# df_union['value_share'] = df_union['spend_local_currency'] / df_union['total_spend_local_currency'] * 100
# df_union['buyers_share'] = df_union['buying_households'] / df_union['total_buying_households'] * 100



#df_union = df_union.drop(columns=['total_spend_local_currency', 'total_buying_households'])
#del totals


# %% variable labels
metrics = {
    'Buying Households': 'Buyers (000 HH)',
    # 'Projected Shoppers': 'Buyers (000 HH)',
    # 'Panel Sample Size': 'Population Raw',
    'Raw Shoppers': 'Raw Buyers',
    'Loyalty Volume Based': 'Loyalty Volume',
    'Loyalty Value Based Local Currency': 'Loyalty Value RUB',
    # 'Loyalty Value Based EUR': 'Loyalty Value EUR',
    'Loyalty Value Based USD': 'Loyalty Value USD',    
    'Percent 2+ Time Buyers': 'Repeat Rate (percent of buyers with frequency 2 and more)',
    'Percent Household Penetration': 'Penetration',
    'Purchase Frequency': 'Frequency',
    'Occasions': 'Trips (000)',
    'Item Buying Rate local currency': 'Spend per buyer RUB',
    # 'Item Buying Rate EUR': 'Spend per buyer EUR',
    'Item Buying Rate USD': 'Spend per buyer USD',   
    'Purchase size local currency': 'Spend per trip RUB',
    # 'Purchase size EUR': 'Spend per trip EUR',
    'Purchase size USD': 'Spend per trip USD',    
    'Purchase size SU': 'Volume per trip SU',
    'Average per SU local currency': 'Average Price per SU (RUB)',
    # 'Average per SU EUR': 'Average Price L/kg/SU/EUR',
    'Average per SU USD': 'Average Price per SU USD',
    'Volume SU': 'Volume SU (000)',
    'Volume Physical Units': 'Volume Packs (000)',
    'Spend Local Currency': 'Value RUB (000)',
    # 'Spend EUR': 'Value EUR',
    'Spend USD': 'Value USD (000)',    
}

metrics_dic = {
    k.lower().replace(' ', '_').replace('+', '_'): v for k, v in metrics.items()
}

metrics_dic.update({
    'value_share': 'Value share, % of Total Demography',
    'buyers_share': 'Buyers share, % of Total Demography'
    })



# %%
#make lbl_dict
product_order = pd.read_excel('codes/feature_groups.xlsx')
demo_order = pd.read_excel('codes/demo_order.xlsx')
segment_order = pd.read_excel('codes/segments_order.xlsx')
shop_order = pd.read_excel('codes/shop_order.xlsx')
period_dic = pd.read_excel('codes/periods.xlsx', sheet_name=None)


lbl_dic = {}
cat_new_codes = {
    '0. Diapers': 1, 
    '0. MALE B&R': 2, 
    '0. Conditioners': 3,
    '0. Shampoos': 4,
    '0. Total Feminine Care': 5, 
    '0. Laundry Detergents': 6
}
lbl_dic['category'] = dict(zip(product_order['category_code'], product_order['Category Name']))
lbl_dic['level_0'] = lbl_dic['category']

lbl_dic['features'] = dict(zip(product_order['feature_code'], product_order['feature']))

lbl_dic['cat_segment'] = dict(zip(segment_order['segment_code'], segment_order['Segment']))

#product_new_codes = dict(zip(product_order['Vendor Product ID'], product_order['product_code']))
lbl_dic['products'] = dict(zip(product_order['product_code'], product_order['full_label']))

lbl_dic['product_hier'] = dict(zip(product_order['product_code'], product_order['product_hier']))

demo_new_codes = dict(zip(demo_order['Buyer Group ID'], demo_order['demo_code']))
lbl_dic['demo_groups'] = dict(zip(demo_order['demo_code'], demo_order['buyers_gr_label']))


lbl_dic['shop_code'] = dict(zip(shop_order['shop_code'], shop_order['position_name_shop']))
lbl_dic['shop_lvls'] = dict(zip(shop_order['shop_lvls'], shop_order['shop_lvls'].astype(str)))
lbl_dic['channel_code'] = dict(zip(shop_order['channel_code'], shop_order['channel_type']))

lbl_dic['product_lvls'] = {k: str(k) for k in product_order['product_lvls']}
lbl_dic['demo_lvls'] = {k: str(k) for k in demo_order['demo_lvls']}

lbl_dic['year'] = dict(zip(period_dic['year']['year_code'], period_dic['year']['year']))
lbl_dic['time_period_type'] = dict(zip(
    period_dic['time_period_type']['time_period_code'], 
    period_dic['time_period_type']['time_period_type']))

lbl_dic['label_num'] = dict(zip(period_dic['period_lbl']['period_code'], period_dic['period_lbl']['label_num']))

lbl_dic['period_lbl'] = dict(zip(period_dic['period_lbl']['period_code'], period_dic['period_lbl']['period_lbl']))

lbl_dic['socdem_gr'] = dict(zip(demo_order['demo_code'] , demo_order['buyers_gr_label'] ))
lbl_dic['demo_groups'] = dict(zip(demo_order['demo_code'] , demo_order['buyers_gr_label'] ))
lbl_dic['demo_hier'] = dict(zip(demo_order['demo_code'] , demo_order['demo_hier'] ))




print("Колонок без словаря",set(df_union.select_dtypes(exclude='float').columns) - set(metrics_dic.keys()|lbl_dic.keys() ))

values_no_code ={} 
for col  in [key for key in lbl_dic if key in df_union.columns]:
    col_codes = set(lbl_dic[col].keys())
    col_values = set(df_union[col].unique())
    non_code_values_col =  col_values - col_codes
    if len(non_code_values_col):
        values_no_code[col] = non_code_values_col
        print(col)

if len(values_no_code):
    print("Колонок с пропусками кодов", len(values_no_code))
    with open('data/tmp/bad_codes.pickle', 'wb') as f:
        pickle.dump(values_no_code, f)   

Diapers
(1578114, 50)
Feminine Care
(2554552, 50)
Hair Conditioners
(3561345, 50)
Laundry Detergents
(7939944, 50)
MALE B&R
(2327033, 50)
Shampoos
(1372281, 50)
Creating united data file complite! (19333269, 50)
Колонок без словаря {'duration', 'segment', 'file', 'position_name_shop'}
label_num
Колонок с пропусками кодов 1


In [16]:
values_no_code ={} 
for col  in [key for key in lbl_dic if key in df_union.columns]:
    col_codes = set(lbl_dic[col].keys())
    col_values = set(df_union[col].unique())
    non_code_values_col =  col_values - col_codes
    if len(non_code_values_col):
        values_no_code[col] = non_code_values_col
        print(col)

label_num


In [22]:
col = 'demo_lvls'
col_codes = set(lbl_dic[col].keys())
col_values = set(df_union[col].unique())
non_code_values_col =  col_values - col_codes
print(non_code_values_col)

set()


In [21]:
col_codes

{1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144}

In [10]:
set(df_union['socdem_gr'].unique())

{np.int64(1),
 np.int64(2),
 np.int64(8),
 np.int64(13),
 np.int64(18),
 np.int64(22),
 np.int64(27),
 np.int64(38),
 np.int64(45),
 np.int64(50),
 np.int64(54),
 np.int64(58)}

In [12]:
set(lbl_dic['socdem_gr'].keys())

{1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63}

In [24]:
with open('data/tmp/bad_codes.pickle', 'rb') as f:
    values_no_code = pickle.load(f)  

In [25]:
values_no_code

{'label_num': {'2MATs: 2021 (10) Oct',
  '2MATs: 2021 (11) Nov',
  '2MATs: 2021 (12) Dec',
  '2MATs: 2022 (01) Jan',
  '2MATs: 2022 (02) Feb',
  '2MATs: 2022 (03) Mar',
  '2MATs: 2022 (04) Apr',
  '2MATs: 2022 (05) May',
  '2MATs: 2022 (06) Jun',
  '2MATs: 2022 (07) Jul',
  '2MATs: 2022 (08) Aug',
  '2MATs: 2022 (09) Sep',
  '2MATs: 2022 (10) Oct',
  '2MATs: 2022 (11) Nov',
  '2MATs: 2022 (12) Dec',
  '2MATs: 2023 (01) Jan',
  '2MATs: 2023 (02) Feb',
  '2MATs: 2023 (03) Mar',
  '2MATs: 2023 (04) Apr',
  '2MATs: 2023 (05) May',
  '2MATs: 2023 (06) Jun',
  '2MATs: 2023 (07) Jul',
  '2MATs: 2023 (08) Aug',
  '2MATs: 2023 (09) Sep',
  '2MATs: 2023 (10) Oct',
  '2MATs: 2023 (11) Nov',
  '2MATs: 2023 (12) Dec',
  '2MATs: 2024 (01) Jan',
  '2MATs: 2024 (02) Feb',
  '2MATs: 2024 (03) Mar',
  '2MATs: 2024 (04) Apr',
  '2MATs: 2024 (05) May',
  '2MATs: 2024 (06) Jun',
  '2MATs: 2024 (07) Jul',
  '2MATs: 2024 (08) Aug',
  '2MATs: 2024 (09) Sep',
  '3MMT: 2021 (10) Oct',
  '3MMT: 2021 (11) Nov',
 